# mvTCR Preprocessing
mvTCR uses a specific format to handle single-cell data, which is based on AnnData objects. If not otherwise stated, we follow the speficition from Scanpy [1] and Scirpy [2]. However, we need some additional information to utilize all functions of mvTCR. In this notebook, we introduce the mvTCR preprocessing pipeline, which adds the required information to the corresponding place in the AnnData object.

All experiments in our paper where conducted on Datasets:
- after Quality Control (cell filtering, doublet detection, ...)
- with normalized and log+1 transformed count data

The pipeline assumes that these steps have already been performed. For further reference, please see Luecken et al [3].

If you know what you are doing: different normalization, log-stabilizing transformations, etc. can also be used, but need to be handled with care!


[1] Wolf, F. A., Angerer, P. & Theis, F. J. Scanpy: large-scale single-cell gene expression data analysis. Genome biology 19, 1–5 (2018).

[2] Sturm, G. et al. Scirpy: a scanpy extension for analyzing single-cell t-cell receptor-sequencing data. Bioinformatics 36, 4817–4818 (2020).

[3] Luecken, M. D. & Theis, F. J. Current best practices in single-cell rna-seq analysis: a tutorial.
Molecular systems biology 15, e8746 (2019).

## Prerequisits

The preprocessing pipeline is showcased on the dataset from Stephenson et al. [4], which can be readily downloaded from:

- https://covid19.cog.sanger.ac.uk/submissions/release1/haniffa21.processed.h5ad
- https://www.ebi.ac.uk/biostudies/files/E-MTAB-10026/TCR_merged-Updated.tsv

and is already quality-controled. 

[4] Stephenson, E. et al. Single-cell multi-omics analysis of the immune response in covid-19. Nature medicine 27, 904–916 (2021).


The mvTCR preprocessing pipeline is taylored for mvTCR-usage and handles the encoding of clonotypes and conditional variables in the required format. However, it is necessary that the adata object is already log-normalized, subsetted to highly variable genes and contains scirpy-encoded TCR information. We demonstrate these steps below.

In [1]:
import scanpy as sc
import scirpy as ir
import pandas as pd
import awkward as ak

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\Jan\.conda\envs\mvtcr_020\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_gex = '../data/Haniffa/haniffa21.processed.h5ad'
path_tcr = '../data/Haniffa/TCR_merged-Updated'

We will load the transcriptome data. To speed up runtime, we will downsample the data to two patients.

In [3]:
adata = sc.read(path_gex)

selected_patients = ['AP1', 'CV0062']
adata = adata[adata.obs['patient_id'].isin(selected_patients)].copy()

Before starting, we take the raw expression counts matrix, total-count normalize it to 10,000 reads per cell to correct for differences in library-size, and logarithmize it:

In [4]:
adata.X = adata.layers['raw']
print(adata.X[0:4,11:20].toarray())
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
print(adata.X[0:4,11:20].toarray())

[[1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1.1596017 0.        0.        0.        0.        1.1596017 0.
  0.        0.       ]
 [0.        0.        0.        0.        0.        1.52811   0.
  0.        0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.        0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.        0.       ]]


To keep the most informative genes, we subset our matrix to the 5000 highest-variable genes. This number can be changed based on the expected variation, and noisy or technical-artifact related genes can be excluded based on prior knowledge.

In [5]:
print('Shape before: ', adata.shape)
sc.pp.highly_variable_genes(adata, n_top_genes=5000)
adata = adata[:, adata.var['highly_variable']]
print('Shape after: ', adata.shape)

Shape before:  (8811, 24929)
Shape after:  (8811, 5000)


Next we add the required TCR information as scirpy formatted covariates in the obs matrix:

In [6]:
df_tcr = pd.read_csv(f'{path_tcr}.tsv', sep='\t')
df_tcr['barcode'] = df_tcr.pop('CellID') # change cell IDs column name to "barcode"
df_tcr = df_tcr[df_tcr['study_id'].isin(selected_patients)] # keep only selected patients
df_tcr.to_csv(f'{path_tcr}.csv')

C:\Users\Jan\AppData\Local\Temp\ipykernel_25604\656390862.py:1: DtypeWarning: Columns (38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tcr = pd.read_csv(f'{path_tcr}.tsv', sep='\t')


In [7]:
adata_tcr = ir.io.read_10x_vdj(f'{path_tcr}.csv') # load with scirpy

barcode2idx = {barcode: i for i, barcode in enumerate(adata_tcr.obs_names)}
idx = [barcode2idx.get(barcode) for barcode in adata.obs_names]

adata.obsm["airr"] = adata_tcr.obsm["airr"][idx]
adata = adata[~ak.is_none(adata.obsm['airr']).to_numpy()]

c:\Users\Jan\.conda\envs\mvtcr_020\lib\site-packages\anndata\utils.py:334: ExperimentalFeatureWarning: Support for Awkward Arrays is currently experimental. Behavior may change in the future. Please report any issues you may encounter!
  warnings.warn(msg, category, stacklevel=stacklevel)
C:\Users\Jan\AppData\Local\Temp\ipykernel_25604\1760237386.py:6: ImplicitModificationWarning: Setting element `.obsm['airr']` of view, initializing view as actual.
  adata.obsm["airr"] = adata_tcr.obsm["airr"][idx]


Lets index the chains

In [8]:
ir.pp.index_chains(adata)

c:\Users\Jan\.conda\envs\mvtcr_020\lib\site-packages\scirpy\pp\_index_chains.py:259: ImplicitModificationWarning: Setting element `.obsm['chain_indices']` of view, initializing view as actual.
  params.adata.obsm[key_added] = ak.zip(res, depth_limit=1)  # type: ignore


mvTCR requires paired data between TCR and GEX. Therefore, we remove all samples without a TRA or TRB CDR3 region.

In [9]:
print(len(adata))
junction_aa = ir.get.airr(adata, "junction_aa")
adata = adata[~(junction_aa['VJ_1_junction_aa'].isna() | junction_aa['VDJ_1_junction_aa'].isna())]
print(len(adata))

5395
4227


## mvTCR preprocessing

In [10]:

#This is only needed if you have not installed mvTCR in your conda environment and want to import the functions locally.
import sys
sys.path.append('..')


In [11]:
from mvtcr.utils_preprocessing import Preprocessing

## All-in-one Pipeline

After we have a fitting dataset containing scirpy-encoded TCR information and expression data we can use mvTCR's preprocessing methods to further bring our data into shape.
The preprocessing pipeline is the fast way to do that your data. 

This features (in order):

- Checks for:
    - Normalization & log transformation checks (experimental)
    - "Reasonable" number of highly variable genes check (500 < n < 5000)
    - Scirpy VDJ gene usage information check
- Encoding of clonotypes
- Encoding of TCR
- One-Hot encoding of conditional variables
- Group-shuffle-splits (into train & validation datasets)

The required parameters and expected outputs of each step are explained in detail in the piece by piece preprocessing section below.

In [12]:
ak.__version__

'2.4.9'

In [13]:
import numpy as np
np.__version__

'1.26.1'

In [15]:
adata

View of AnnData object with n_obs × n_vars = 4227 × 5000
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap', 'log1p', 'chain_indices'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap', 'airr', 'chain_indices'
    layers: 'raw'

In [14]:
Preprocessing.preprocessing_pipeline(adata, 
                                     clonotype_key_added='clonotype', 
                                     airr_name='junction_aa',
                                     cond_vars=['patient_id'],
                                     group_col='clonotype', 
                                     val_split=0.2)

TypeError: no numpy.add overloads for custom types: AwkwardArrayView, AwkwardArrayView

This error occurred while calling

    numpy.add.__call__(
        <AwkwardArrayView [1, 1, 1, 1, 1, ..., 1, 1, 1, 1, 1] type='4227 * ...'>
        <AwkwardArrayView [1, 1, 1, 1, 2, ..., 1, 1, 1, 1, 1] type='4227 * ...'>
    )

In [ ]:
adata.obs.set.value_counts()

# Piece by Piece Preprocessing

All the features inside the pipeline can be executed seperately as well, to perform a step-by-setp preprocessing or only specific methods.

Make sure to freshly load the data if you have used the pipeline.

### Checking if adata is in a mvTCR compatible shape

In [16]:
Preprocessing.check_if_valid_adata(adata)

True

### Encoding clonotypes with Scirpy

For training the shared embedding, we advise oversampling rare clonotypes. This avoids the model overfitting to few selected TCR sequences from highly expanded clonotypes. Therefore, we need to add a clonotype label to adata.obs. Here, we define a unique clonotype via Scirpy as having exactly the same CDR3 sequence in TRA and TRB chains.

In [17]:
Preprocessing.encode_clonotypes(adata, key_added='clonotype')

adata.obs.clonotype.value_counts()

TypeError: no numpy.add overloads for custom types: AwkwardArrayView, AwkwardArrayView

This error occurred while calling

    numpy.add.__call__(
        <AwkwardArrayView [1, 1, 1, 1, 1, ..., 1, 1, 1, 1, 1] type='4227 * ...'>
        <AwkwardArrayView [1, 1, 1, 1, 2, ..., 1, 1, 1, 1, 1] type='4227 * ...'>
    )

### Adding TCR encoding

Next, we encode the TCR sequence numerically to adata.obsm. Here, we need to provide the name of the column storing the CDR3a and CDR3b. Additionally, we need to specificy the padding paremter (which if set to None uses the maximal CDR3 sequence length as default). If you plan to add new data in the future via a pretrained model, you might want to add some safety margin.

In [18]:
Preprocessing.encode_tcr(adata=adata, 
                         airr_name='junction_aa', 
                         alpha_label_key='alpha_seq', 
                         alpha_length_key='alpha_len',
                         beta_label_key='beta_seq', 
                         beta_length_key='beta_len')

adata.obsm['beta_seq'][:5]

array([[ 2,  1,  1, 18, 12, 12, 12,  1,  6, 12, 11, 10, 17,  5,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 2,  1, 18,  3, 17,  6,  6,  6, 12,  9, 10, 17,  5,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 2, 18, 18, 11, 13,  5,  6,  6, 20, 14,  9, 18, 17,  5,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 2,  8, 18, 15,  6, 18, 16, 15,  1, 15, 10, 11,  5,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 2,  1,  1, 13, 17, 16,  6, 17, 20,  9, 20,  8,  5,  0,  0,  0,
         0,  0,  0,  0,  0,  0]])

### Adding conditional variables

Conditioning your model partially removes the effect from a specified condition. We can add conditional variables for e.g. donor, to avoid batch effects over multiple samples.

In [19]:
Preprocessing.encode_conditional_var(adata, column_id='patient_id')
adata.obsm['patient_id']

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

### Creating training and validation splits

The splitting improves the data spliting into training and validation sets by two properties:
- Stratified splitting: balance a label of interest (normally a variable to be predicted, e.g. antigen specificity) so the label distribution is roughly the same in both sets.
- Avoid training data leakage into validation: used for clonotypes, to ensure that each clonotype is observed only during training or validation.

In [20]:
train, val = Preprocessing.stratified_group_shuffle_split(adata, stratify_col='full_clustering', group_col='clonotype', val_split=0.2, random_seed=42)

adata.obs['set'] = 'train'
adata.obs.loc[val.index, 'set'] = 'val'

adata.obs.set.value_counts()

  0%|          | 0/23 [00:00<?, ?it/s]


KeyError: 'clonotype'

Alternatively group splitting is available by itself with:

In [ ]:
gtrain, gval = Preprocessing.group_shuffle_split(adata, group_col='clonotype', val_split=0.2, random_seed=42)

adata.obs['set'] = 'train'
adata.obs.loc[gval.obs.index, 'set'] = 'val'

adata.obs.set.value_counts()

set
train    3384
val       843
Name: count, dtype: int64

### Finish. You are all set and done to use mvTCR! Save your data!

In [20]:
path_out = '../data/preprocessed/haniffa_test_new.h5ad'
adata.write_h5ad(path_out, compression='gzip')